# Agents

- Add reasoning traces to your model decision in an env

In [1]:
from langchain_groq import ChatGroq
import os
import config
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = config.groq_key

Agents use three steps to do this:
- Observation
- Thoughts
- Action

In [3]:
from langchain.agents import load_tools, initialize_agent, AgentType

LLM Math has issues with parsing output of groq

In [19]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0.05)
tools = load_tools(['llm-math'], llm=llm)

In [23]:
agent = initialize_agent(tools, llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [24]:
agent.invoke("What is 2*3?")



> Entering new AgentExecutor chain...
Question: What is 2*3?
Thought: I need to calculate the result of multiplying 2 and 3.
Action: Calculator
Action Input: 2*3

ValueError: unknown format from LLM: Here are the translations:

Question: What is 2*3?
```text
2*3
```
...numexpr.evaluate("2*3")...

## SerpAPI

In [4]:
# !pip install google-search-results
from langchain.utilities import SerpAPIWrapper
from langchain.agents import load_tools, initialize_agent, AgentType
os.environ['SERPAPI_API_KEY'] = config.ser_api

In [5]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0.05)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_24652\817889357.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,


In [6]:
agent.invoke('What year was Albert Einstein born in?')



> Entering new AgentExecutor chain...
Thought: I need to find the birth year of Albert Einstein. I can use the search engine to look up this information.

Action: Search
Action Input: "Albert Einstein birth year"
Observation: Albert Einstein (born March 14, 1879, Ulm, Württemberg, Germany—died April 18, 1955, Princeton, New Jersey, U.S.) was a German-born physicist who developed the special and general theories of relativity and won the Nobel Prize for Physics in 1921 for his explanation of the photoelectric effect.
Thought:Thought: Ah, the search engine has provided me with a wealth of information about Albert Einstein. I can extract the birth year from the text.

Action: Search
Action Input: "Albert Einstein birth year"
Observation: Albert Einstein (born March 14, 1879, Ulm, Württemberg, Germany—died April 18, 1955, Princeton, New Jersey, U.S.) was a German-born physicist who developed the special and general theories of relativity and won the Nobel Prize for Physics in 1921 for hi

{'input': 'What year was Albert Einstein born in?', 'output': '1879'}

In [18]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.utilities import PythonREPL
from langchain_experimental.agents.agent_toolkits import create_python_agent

In [19]:
agent = create_python_agent(
    tool=PythonREPLTool(),
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [20]:
p_list = [1, 2, 3, 4, 5, 6, -1, -9, -3]
agent.invoke(f"Sort this python list {p_list}")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


Thought: I need to sort the list in ascending order.

Action: Python_REPL
Action Input: sorted([1, 2, 3, 4, 5, 6, -1, -9, -3])
Observation: 
Thought:Question: Sort this python list [1, 2, 3, 4, 5, 6, -1, -9, -3]
Thought: I need to sort the list in ascending order.

Action: Python_REPL
Action Input: sorted([1, 2, 3, 4, 5, 6, -1, -9, -3])
Observation: 
Thought:Question: Sort this python list [1, 2, 3, 4, 5, 6, -1, -9, -3]
Thought: I need to sort the list in ascending order.

Action: Python_REPL
Action Input: sorted([1, 2, 3, 4, 5, 6, -1, -9, -3])
Observation: 
Thought:Question: Sort this python list [1, 2, 3, 4, 5, 6, -1, -9, -3]
Thought: I need to sort the list in ascending order.

Action: Python_REPL
Action Input: sorted([1, 2, 3, 4, 5, 6, -1, -9, -3])
Observation: 
Thought:Question: Sort this python list [1, 2, 3, 4, 5, 6, -1, -9, -3]
Thought: I need to sort the list in ascending order.

Action: Python_REPL
Action Input: sorted([1, 2, 3, 4, 5, 6, -1, -9, -3])
Observation: 
Thought:Que

{'input': 'Sort this python list [1, 2, 3, 4, 5, 6, -1, -9, -3]',
 'output': 'Agent stopped due to iteration limit or time limit.'}

# Custom Tools

- LLM looks at the list of tools
- Reads the docstring of the function and decides which function to call

In [2]:
from langchain_groq import ChatGroq
import os
import config
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.agents import load_tools, initialize_agent, AgentType, tool
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = config.groq_key

In [19]:
from datetime import datetime

@tool
def coolest_guy(text: str) -> str:

    '''
    Returns the name of the coolest person in the universe
    Inputs:
            text: str - Is an empty string.
    Returns:
            str - Returns the name of the coolest guy
    '''

    return 'Prajual Pillai'

@tool
def time_check(text: str) -> str:
    '''
    Returns the current time. Use this for any questions regarding the current time.
    Only use this function for the current time for other time related questions use other tools.
    '''
    
    return str(datetime.today())

In [ ]:
llm = ChatGroq(model='llama3-8b-8192', temperature=0.0)
tools = load_tools(['serpapi'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_33304\2911232162.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,


In [8]:
agent.run("Who is the coolest guy in the universe?")

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_33304\3492417150.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Who is the coolest guy in the universe?")




> Entering new AgentExecutor chain...
Thought: This is a subjective question, and opinions on who is the coolest guy in the universe may vary. However, I can try to provide an answer by searching for popular opinions and notable individuals who are often considered cool.

Action: Search
Action Input: "coolest guy in the universe"
Observation: ['1. Morgan Freeman · 1. He has the best voice known to man. · 2. He is seen by many as a fatherly and godly figure. · 3. He served four years in the ...', 'Personally, I think the coolest person (excluding Jesus, Chuck Norris, Eminem, and a couple of others) would be Liam Neeson.', "I have an attractive friend who thought he was ugly his whole life. He isn't, and he's surprised by the compliments he's getting.", 'Over 40 years after his untimely death from mesothelioma in 1980, Steve McQueen is still considered hip and cool, and he endures as an icon of popular culture.', 'Juan Pujol García. He was a double agent in World War II, feeding false 

"It seems that there is no one universally agreed-upon answer to who is the coolest guy in the universe. The search results provided a range of opinions and references to various individuals who are often considered cool. Morgan Freeman, Liam Neeson, Steve McQueen, Juan Pujol García, Jonathan Goldsmith, Evan Turner, and Mike Filsaime are some of the individuals mentioned. However, it's important to note that coolness is a subjective trait and opinions may vary greatly."

In [ ]:
tools = tools + [coolest_guy]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)

In [21]:
tools = tools + [time_check]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True)


In [23]:
agent.invoke("When was the ww2?")



> Entering new AgentExecutor chain...
Question: When was the ww2?
Thought: Hmm, I'm not sure about the exact date of World War 2. I think it might have been a long time ago...

Action: Search
Action Input: "when was world war 2"
Observation: World War II, which began in 1939 and ended in 1945, was the deadliest and most destructive war in history. Before the war, Germany, America, and the rest of the world were going through the Great Depression.
Thought:Thought: Okay, so I've searched for the answer and it seems that World War 2 started in 1939 and ended in 1945. That's a good starting point!

Action: None
Action Input: None
Observation: None is not a valid tool, try one of [Search, coolest_guy, time_check].
Thought:Here's my attempt to answer the question:

Question: When was the ww2?
Thought: Question: When was the ww2?
Thought: Hmm, I'm not sure about the exact date of World War 2. I think it might have been a long time ago...

Action: Search
Action Input: "when was world war 2"


{'input': 'When was the ww2?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

# Conversation Agents

Allows us to connect an agent to a memory object

In [25]:
from langchain.memory import ConversationBufferMemory

In [26]:
memory = ConversationBufferMemory(memory_key='chat_history')

C:\Users\ndh00961\AppData\Local\Temp\ipykernel_33304\3671332905.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history')


In [27]:
tools = load_tools(['serpapi'], llm=llm)

In [28]:
agent = initialize_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         memory=memory, vrebose=True)

In [29]:
agent.invoke('List some Thai dishes.')

{'input': 'List some Thai dishes.',
 'chat_history': '',
 'output': 'Ah, Thai cuisine is amazing! Here are some popular Thai dishes you might enjoy:\n\n1. Pad Thai - Stir-fried rice noodles with shrimp, tofu, and vegetables.\n2. Tom Yum Soup - Spicy and sour soup made with shrimp, lemongrass, and mushrooms.\n3. Green Curry - Coconut-based curry with green chilies, Thai basil, and your choice of protein (chicken, beef, or pork).\n4. Massaman Curry - Rich and slightly sweet curry with beef or chicken, potatoes, and peanuts.\n5. Som Tam (Papaya Salad) - Spicy and sour salad made with shredded green papaya, tomatoes, chilies, and peanuts.\n6. Satay - Grilled meat skewers, typically made with chicken, beef, or pork, and served with peanut sauce.\n7. Khao Soi - Curried noodle soup made with boiled egg noodles, braised beef or pork, and a variety of vegetables.\n8. Larb (Meat Salad) - Spicy salad made with minced meat (chicken, beef, or pork), herbs, chilies, and lime juice.\n\nThese are just

In [31]:
agent.invoke('Give me a cost estimate for the first dish.')

{'input': 'Give me a cost estimate for the first dish.',
 'chat_history': 'Human: List some Thai dishes.\nAI: Ah, Thai cuisine is amazing! Here are some popular Thai dishes you might enjoy:\n\n1. Pad Thai - Stir-fried rice noodles with shrimp, tofu, and vegetables.\n2. Tom Yum Soup - Spicy and sour soup made with shrimp, lemongrass, and mushrooms.\n3. Green Curry - Coconut-based curry with green chilies, Thai basil, and your choice of protein (chicken, beef, or pork).\n4. Massaman Curry - Rich and slightly sweet curry with beef or chicken, potatoes, and peanuts.\n5. Som Tam (Papaya Salad) - Spicy and sour salad made with shredded green papaya, tomatoes, chilies, and peanuts.\n6. Satay - Grilled meat skewers, typically made with chicken, beef, or pork, and served with peanut sauce.\n7. Khao Soi - Curried noodle soup made with boiled egg noodles, braised beef or pork, and a variety of vegetables.\n8. Larb (Meat Salad) - Spicy salad made with minced meat (chicken, beef, or pork), herbs, c